In [18]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,multilabel_confusion_matrix,roc_auc_score,roc_curve,auc,ConfusionMatrixDisplay

In [3]:
x_train_all=pd.read_csv("CompleteDataSet/x_train_all.csv")
y_train_all=pd.read_csv("CompleteDataSet/y_train_all.csv")
x_test_all=pd.read_csv("CompleteDataSet/x_test_all.csv")
y_test_all=pd.read_csv("CompleteDataSet/y_test_all.csv")

### Metrics

In [26]:
def metric(Y_test,predicted):   
    accuracy = accuracy_score(Y_test,predicted )
    f1 = f1_score(predicted, Y_test, average="weighted")
    conf_matrix = confusion_matrix(predicted, Y_test)
    precision = precision_score(Y_test, predicted, average="weighted",zero_division=1)
    recall = recall_score(Y_test, predicted, average="weighted")

    # Y_test_bin = label_binarize(Y_test, classes=[0, 1, 2,3,4,5,6,7,8,9])
    # roc = roc_auc_score(Y_test_bin, predicted_probs, multi_class="ovr", average="weighted")

    conf_matrices = multilabel_confusion_matrix(Y_test, predicted)
    tp = conf_matrices[:,1,1]
    fp = conf_matrices[:,0,1]
    tn = conf_matrices[:,0,0]
    fn = conf_matrices[:,1,0]
    # fpr = (fp/(tn + fp), tp + fn)
    tnr = (tn/(tn + fp), tp + fn)

    # fpr, tpr, _ = roc_curve(Y_test_bin.ravel(), predicted_probs.ravel())


    # auc_val = auc(fpr, tpr)
    # plt.figure()
    # plt.plot(fpr, tpr, color='darkorange', lw=2, label='AUC (area = %0.2f)' % auc_val)
    # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('ROC Curve')
    # plt.legend(loc="lower right")
    # plt.show()

    # out=ConfusionMatrixDisplay(conf_matrix,display_labels=gnb.classes_)
    # out.plot()
    # plt.show()
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall/Sensitivity/True Positive Rate:", recall)
    print("Specificity:", tnr)
    # print("False Positive Rate:", fpr)
    # print("Area under ROC curve:", roc)
    print("Confusion Matrix:\n", conf_matrix)


### Cross Validation

In [4]:
from sklearn.model_selection import KFold 


In [5]:
def kfold_10(input,output):
    kf = KFold(n_splits=10) 
    dt_classifier = DecisionTreeClassifier()
    accuracy_scores = []
    for train_index, test_index in kf.split(input):
        print("Train:", len( train_index), "Validation:",len(test_index))
        X_train, X_test = input.iloc[train_index], input.iloc[test_index] 
        y_train, y_test = output.iloc[train_index], output.iloc[test_index]
        dt_classifier.fit(X_train, y_train)
        pred=dt_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        accuracy_scores.append(accuracy)
    average_accuracy = sum(accuracy_scores) / 10

    print(f"Average Accuracy: {average_accuracy}")



In [6]:
x_train_all.shape

(9690, 2304)

In [7]:
y_train_all.shape

(9690, 1)

In [8]:
kfold_10(x_train_all,y_train_all)

Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Train: 8721 Validation: 969
Average Accuracy: 0.4893704850361197


In [24]:
def train_test_split(X_train,X_test,y_test,y_train):
    dt_classifier = DecisionTreeClassifier()
    dt_classifier.fit(X_train,y_train)
    pred=dt_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    metric(y_test,pred)
    print("The Accuracy is:",accuracy)

In [27]:
train_test_split(x_train_all,x_test_all,y_test_all,y_train_all)

Accuracy: 0.6841423948220064
F1 Score: 0.6880645350053534
Precision: 0.6795979099496501
Recall/Sensitivity/True Positive Rate: 0.6841423948220064
Specificity: (array([0.98514851, 0.90590717, 0.91153846, 0.94090909, 0.92592593,
       0.99174917, 0.98633333, 0.99042904, 0.98707483, 0.98933333]), array([ 60, 720, 750, 450, 660,  60,  90,  60, 150,  90], dtype=int64))
Confusion Matrix:
 [[ 16  11   2   2  28   0   0   0   2   0]
 [ 11 566  70  24  58   7   8  17  24   4]
 [ 10  50 569  63  34   1   4   4  19  22]
 [  2  18  42 300  49   7  13   7  13   5]
 [ 15  55  57  31 470  15   3   0   4   0]
 [  0   1   0   0   1  30   8   9   5   1]
 [  3   0   3  14  10   0  36   0  10   1]
 [  0   9   3   3   3   0   0  18   6   5]
 [  3   7   2   3   3   0  13   0  64   7]
 [  0   3   2  10   4   0   5   5   3  45]]
The Accuracy is: 0.6841423948220064
